In [15]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colormaps
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import json

import xtrack as xt
import xpart as xp
import xobjects as xo
import xcoll as xc

In [16]:
def remove_offmom_bpms_apers(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2):
    "Remove apertures of off-momentum BPMs which give flanges as bottlenecks"
    tw = line.twiss()
    tt = line.get_table()
    mask_disp = 5*np.sqrt(tw.betx*exn*pmass/nrj)+n_buckets*bucket_height*tw.dx > 0.025
    mask_bpm = ['bp' in name for name in tt.name]
    mask_aper = np.array(['aper' in name for name in tt.name])
    offmom_bpms = tt.name[mask_disp & mask_bpm & ~mask_aper]
    aper_to_remove = [f'{name}{suffix}' for name in offmom_bpms for suffix in ('.a_aper', '.b_aper')]
    line.remove(aper_to_remove)

In [17]:
line = xt.Line.from_json('../../injection_lines/sps_with_aperture_inj_q20_beam_sagitta4.json')
tt = line.get_table()
tw = line.twiss()
env = line.env
cavity_elements, cavity_names = line.get_elements_of_type(xt.Cavity)

for name in cavity_names:
    line[name].frequency = 200e6
    line[name].lag = 180
line['acl.31735'].voltage = 0 #setting 800 cav to 0V
line['actcse.31632'].voltage = 3.0e6

remove_offmom_bpms_apers(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2)

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [18]:
# # Adding errors
# tte = env.elements.get_table()
# mask_rbends = tte.element_type == 'RBend'
# mask_quads = tte.element_type == 'Quadrupole'
# mask_sextupoles = tte.element_type == 'Sextupole'

# mba = tte.rows[mask_rbends].rows['mba.*'].name
# mbb = tte.rows[mask_rbends].rows['mbb.*'].name
# qf = tte.rows[mask_quads].rows['qf.*'].name
# qd = tte.rows[mask_quads].rows['qd.*'].name
# lsf = tte.rows[mask_sextupoles].rows['lsf.*'].name
# lsd = tte.rows[mask_sextupoles].rows['lsd.*'].name

# env.vars['qph_setvalue'] = 0.0
# env.vars['qpv_setvalue'] = 0.0

# # Set the strengths according to Hannes' measurements
# for nn in mba:
#     env[nn].knl = np.array([0., 0., 2.12e-3, 0., -5.74, 0.])

# for nn in mbb:
#     env[nn].knl = np.array([0., 0., -3.19e-3, 0., -5.10, 0.])

# for nn in qf:
#     env[nn].knl = np.array([0., 0., 0., 0.75e-1, 0., -0.87e3])

# for nn in qd:
#     env[nn].knl = np.array([0., 0., 0., -2.03e-1, 0., 2.04e3])

In [19]:
env.vars['qph_setvalue'] = 0.5
env.vars['qpv_setvalue'] = 0.5
qx = 20.13
qy = 20.18
opt = line.match(
    method='6d', # <- passed to twiss
    vary=[
        xt.VaryList(['kqf0', 'kqd0'], step=1e-8, tag='quad'),
        xt.VaryList(['qph_setvalue', 'qpv_setvalue'], step=1e-4, tag='sext'),
    ],
    targets = [
        xt.TargetSet(qx=qx, qy=qy, tol=1e-6, tag='tune'),
        xt.TargetSet(dqx=0.5*qx, dqy=0.5*qy, tol=1e-2, tag='chrom'),
    ])

                                             
Optimize - start penalty: 0.1747                            
Matching: model call n. 7 penalty = 1.7532e-04              
Optimize - end penalty:  0.000175324                            


In [20]:
num_particles  = 1000
line_type = 'linear'
plane = 'DPpos'  # 'DPpos' or 'DPneg'
sweep = 6000
sweep = -abs(sweep) if plane == 'DPpos' else abs(sweep)
num_turns = 6000
nemitt_x = 2e-6
nemitt_y = 2e-6
sigma_z = 0.224

In [21]:
tidp_ap_tot = 147
block_mvt = 29

line.discard_tracker()
tidp = xc.EverestCollimator(length=4.3, material=xc.materials.Carbon, jaw_L= tidp_ap_tot/2 + block_mvt, jaw_R = -tidp_ap_tot/2 + block_mvt)
line.collimators.install(names=['tidp.11434'], elements=[tidp])

tw = line.twiss()
line.build_tracker()

Slicing line:   0%|          | 0/32246 [00:00<?, ?it/s]

The line already has an associated tracker


In [22]:
part = xp.generate_matched_gaussian_bunch(nemitt_x=nemitt_x,
                                          nemitt_y=nemitt_y,
                                          sigma_z=sigma_z, num_particles=num_particles, line=line)

*** Maximum RMS bunch length 0.2361011040043429m.
... distance to target bunch length: -2.2226e-01
... distance to target bunch length: 5.5784e-03
... distance to target bunch length: 5.2361e-03
... distance to target bunch length: -6.3267e-03
... distance to target bunch length: 1.9948e-03
... distance to target bunch length: -3.8258e-04
... distance to target bunch length: 5.3833e-05
... distance to target bunch length: 1.2594e-06
... distance to target bunch length: -1.0995e-10
... distance to target bunch length: 1.2805e-07
--> Bunch length: 0.22399999989004857
--> Emittance: 0.2797155229422308


In [23]:
line.discard_tracker()
line.build_tracker(_context=xo.ContextCpu(omp_num_threads='auto'))

rf_sweep = xc.RFSweep(line)
rf_sweep.prepare(sweep_per_turn=sweep/num_turns)
rf_sweep.info()

line.scattering.enable()
line.track(particles=part, num_turns=num_turns, time=True, with_progress=5)
line.scattering.disable()
print(f"Done sweeping RF in {line.time_last_track:.1f}s.")

Compiling ContextCpu kernels...


ld: warning: duplicate -rpath '/Users/lisepauwels/miniforge3/envs/xcoll/lib' ignored
ld: warning: duplicate -rpath '/Users/lisepauwels/miniforge3/envs/xcoll/lib' ignored


Done compiling ContextCpu kernels.


Slicing line:   0%|          | 0/32246 [00:00<?, ?it/s]

Compiling ContextCpu kernels...


...

Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...


ld: warning: duplicate -rpath '/Users/lisepauwels/miniforge3/envs/xcoll/lib' ignored
ld: warning: duplicate -rpath '/Users/lisepauwels/miniforge3/envs/xcoll/lib' ignored


Done compiling ContextCpu kernels.
Enabled time-dependent variables in the line.
The current frequency is 199999999.0Hz, adding -1.0Hz per turn.
This sweep will move the center of the bucket with Δδ = 2.788e-06 per turn.
The bucket height is 0.002982, so this implies the sweep will shift one bucket every 2139.11 turns.


Tracking:   0%|          | 0/6000 [00:00<?, ?it/s]

Done sweeping RF in 465.6s.


In [24]:
idxs, counts = np.unique(part.at_element, return_counts=True)
dico = {line.element_names[idx]: int(counts[i]) for i, idx in enumerate(idxs)}

In [25]:
with open(f'LossMaps/{line_type}_{plane}.json', 'w') as f:
    json.dump(dico, f, indent=4)

In [26]:
dico

{'sps$start': 1,
 'vcak.21101.a_aper': 144,
 'vtto.31105.a_aper': 140,
 'vcak.41101.a_aper': 11,
 'vcak.51101.a_aper': 36,
 'vcak.61101.a_aper': 668}

In [27]:
tw.rows['drift_0..0>>10':'drift_0..0<<10']

TwissTable: 32228 rows, 70 cols
name                           s             x            px             y            py ...
mba.10030..1             4.07099  -9.34028e-06   1.81921e-07  -2.97267e-06  -9.21034e-08
mba.10030..2_aper2       4.69699  -9.22674e-06    1.8082e-07  -3.03033e-06  -9.21034e-08
mba.10030..2             4.69699  -9.22674e-06    1.8082e-07  -3.03033e-06  -9.21034e-08
mba.10030..3_aper2         5.323  -9.11389e-06   1.79718e-07  -3.08799e-06  -9.21034e-08
mba.10030..3               5.323  -9.11389e-06   1.79718e-07  -3.08799e-06  -9.21034e-08
mba.10030..4_aper2         5.949  -9.00173e-06   1.78617e-07  -3.14564e-06  -9.21034e-08
mba.10030..4               5.949  -9.00173e-06   1.78617e-07  -3.14564e-06  -9.21034e-08
mba.10030..5_aper2         6.575  -8.89026e-06   1.77515e-07   -3.2033e-06  -9.21034e-08
mba.10030..5               6.575  -8.89026e-06   1.77515e-07   -3.2033e-06  -9.21034e-08
mba.10030..6_aper2         7.201  -8.77948e-06   1.76413e-07  -3.26096e-06

In [28]:
tw.rows['drift_0..0<<1':'drift_0..0>>1']

TwissTable: 3 rows, 70 cols
name                          s             x            px             y            py ...
qf.10010                      0  -9.55486e-06  -1.60954e-07  -2.75274e-06   7.65175e-09
drift_0..0                3.085  -9.52041e-06   1.83076e-07  -2.88185e-06    -9.212e-08
veqf.10010.b_aper        3.2504  -9.49013e-06   1.83076e-07  -2.89709e-06    -9.212e-08

The main loss location is the same for both the linear line and the error line (with aperture)

In [1]:
ll = 'sps_errors_chroma_-0.1.json'
ll.split('.json')[0]

'sps_errors_chroma_-0.1'